<div style="background-color: #2C3E50; color: #CCCCCC; padding: 20px; border-radius: 10px;">

# **Borrowing Simulation on Halalend Protocol**
#### In this notebook, we'll explore how borrowing works within the Halalend lending protocol. Imagine you need a short-term loan and you have an asset that you can use as collateral. We'll walk through how the protocol assesses the loan request, determines how much you can borrow, and ensures the safety of the loan against market fluctuations.

## Assuming we have the following **Protocol Parameters:**

</div>


In [5]:
// Collateral and Loan Settings
let minimum_collateral_ratio = 2.0  // Minimum Collateral Ratio for ADA as Asset A
let minimum_collateral_share_assetb = 0.1 // Minimum percentage required for collateral in Asset B
let liquidationThreshold = 1.3  // Collateral Ratio at which liquidation starts 

// Loan Settings
let minimum_loan_amount = 100.0  // USD, minimum amount one can borrow
let maximum_loan_term_in_milliseconds = 1_209_600_000  // Maximum term for a loan in milliseconds

// Financial and Risk Management Settings
let liquidation_fee = 10.0  // USD, fee charged upon liquidation
let protocol_usage_fee = 5.0  // USD, fee for using the protocol per loan operation

<div style="background-color: #2C3E50; color: #CCCCCC; padding: 20px; border-radius: 10px;">


# **Loan Eligibility and Processing Calculations**

#### In this section, we explore the core calculations that underpin our lending protocol. These calculations ensure that loans are issued safely, maintaining the integrity and solvency of the protocol.

---

### **Current Collateral Ratio**

##### The Current *Collateral Ratio* is fundamental in assessing the safety of a loan. It compares the total value of the collateral to the loan amount, ensuring that the loan is adequately secured.

```
calculate_collateral_ratio a au b bu c cu = ((a * au) + (b * bu)) / (c * cu)
```

#### Here, *a* and *b* represent the quantities of Asset A and Asset B, while *au* and *bu* are their respective unit values in USD. *c* is the loan amount, and *cu* is the currency unit value (typically 1 if the loan is in USD).
---

### **Minimum Required Asset B Collateral**

##### *Asset B*, our protocol's utility token, must constitute a minimum share of the collateral. This calculation ensures that this requirement is met.

```
calculate_minimum_assetb_collateral loan_amount_usd = loan_amount_usd * minimum_collateral_ratio * minimum_collateral_share_assetb
```
---

### **Maximum Loanable Amount**

##### Determining the maximum amount that can be loaned against a given collateral ensures that loans remain within safe limits, preserving the protocol's health.

```
calculate_max_loanable_amount collateral_usd = collateral_usd * (1.0 / minimum_collateral_ratio)
```
---

### **Asset B Collateral Requirement**

##### This check confirms whether the collateral in Asset B meets the protocol's minimum requirements, a key factor in the loan approval process.

```
is_assetb_collateral_met loan_amount_usd utility_asset_usd = utility_asset_usd >= (loan_amount_usd * minimum_collateral_ratio * minimum_collateral_share_assetb)
```
---

### **Total Minimum Collateral Requirement**

##### It's crucial to verify that the combined collateral meets or exceeds the minimum collateral ratio, ensuring the loan's security against market volatility.

```
is_collateral_ratio_met a au b bu c cu = (calculate_collateral_ratio a au b bu c cu) >= minimum_collateral_ratio
```
---

### **Eligiblity Check**

##### Finally, assessing whether a loan can be issued involves a comprehensive check, including collateral ratios, minimum amounts, and loan terms.

```
can_loan a au b bu c cu loan_term = collateral_ratio_met && minimum_collateral_asset_b_met && minimum_loan_amount_met && loan_term_met
```
---


</div>

In [6]:
// Calculate the current collateral ratio
let calculate_collateral_ratio a au b bu c cu = ((a * au) + (b * bu)) / (c * cu)

// Calculate the minimum required asset B collateral
let calculate_minimum_assetb_collateral loan_amount_usd =
    loan_amount_usd * minimum_collateral_ratio * minimum_collateral_share_assetb

// Calculate the max loanable amount for a given collateral
let calculate_max_loanable_amount collateral_usd =
    let max_loan_to_value = 1.0 / minimum_collateral_ratio
    collateral_usd * max_loan_to_value

// Calculate if the minimum utility token collateral is met
let is_assetb_collateral_met loan_amount_usd utility_asset_usd =
    let required_collateral = loan_amount_usd * minimum_collateral_ratio
    let required_asset_b_collateral = required_collateral * minimum_collateral_share_assetb
    utility_asset_usd >= required_asset_b_collateral

// Calculate if the total minimum collateral is met
let is_collateral_ratio_met a au b bu c cu =
    let collateral_ratio = calculate_collateral_ratio a au b bu c cu
    collateral_ratio >= minimum_collateral_ratio

// Calculate if the loan can be taken
let can_loan a au b bu c cu loan_term =
    let collateral_ratio_met = is_collateral_ratio_met a au b bu c cu
    let minimum_collateral_asset_b_met = is_assetb_collateral_met (c * cu) (b * bu)
    let minimum_loan_amount_met = (c * cu) >= minimum_loan_amount
    let loan_term_met = loan_term <= maximum_loan_term_in_milliseconds

    printf "LOAN CONDITIONS\n"
    printf "Minimum Collateral Ratio Met: %b\n" collateral_ratio_met
    printf "Minimum Collateral Asset B Met: %b\n" minimum_collateral_asset_b_met
    printf "Minimum Loan Amount Met: %b\n" minimum_loan_amount_met
    printf "Loan Term Met: %b\n\n" loan_term_met

    collateral_ratio_met && minimum_loan_amount_met && loan_term_met && minimum_collateral_asset_b_met

<div style="background-color: #2C3E50; color: #CCCCCC; padding: 20px; border-radius: 10px;">

## <span style="color: #CCCCCC;">Creating a Loan Position: User's Guide</span>

#### When a user decide to avail a loan from our protocol, here's the essential information they'll need to provide and how it's used to evaluate the loan request.

---

### **Loan Request Details**

#### First, let's break down the key components of the loan request:

- *Loan Amount:* The total sum you wish to borrow. For our example, you're looking to borrow $120.0.
- *Loan Exchange Rate:* The value of the loan currency against USD.
- *Collateral:* Assets pledged to secure the loan in Asset A and Asset B (Protocol's Utility Token).
- *Exchange Rates for Collateral:* The current market value of the collateral assets in USD.
- *Loan Term:* The duration for which the user intends to borrow the funds, set at 1,209,600,000 milliseconds (equivalent to 14 days).


### *Evaluation Process*

#### The protocol evaluates the loan request based on the following criteria:

### **Collateral Valuation:**  First, we convert the collateral quantities into USD using the provided exchange rates:

```
total_collateral_value = (asset_a_collateral * asset_a_exchange_rate) + (asset_b_collateral * asset_b_exchange_rate)
```

### **Current Collateral Ratio (CR):**  We assess the safety of the loan by calculating the CR, ensuring it exceeds the minimum threshold set by the protocol:

```
current_collateral_ratio = total_collateral_value / (loan_amount * loan_exchange_rate)
```

### **Asset B Minimum Requirement:**  We verify that Asset B's contribution meets the 10% minimum collateral requirement:

```
asset_b_value_usd = asset_b_collateral * asset_b_exchange_rate
minimum_required_asset_b = (loan_amount * loan_exchange_rate) * minimum_collateral_ratio * minimum_collateral_share_assetb
```

#### Asset B's value in USD must exceed minimum_required_asset_b to satisfy this requirement.

### **Loan Term Validation:**  Lastly, we ensure the requested loan term does not exceed the maximum allowed duration.


</div>




## <span style="color: #CCCCCC;">Loan Details</span>

In [7]:
// amount to borrow and exchange rate
let loan_amount = 120.0
let loan_exchange_rate = 1.0

// amount of collateral and exchange rate
let asset_a_collateral = 350.0
let asset_a_exchange_rate = 0.5
let asset_b_collateral = 500.0
let asset_b_exchange_rate = 0.02

// loan term
let loan_term = 1_209_600_000


## <span style="color: #CCCCCC;">Actual Calculations</span>

In [8]:
// calculate usd value
let asset_a_usd = asset_a_collateral * asset_a_exchange_rate
let asset_b_usd = asset_b_collateral * asset_b_exchange_rate
let total_collateral = asset_a_usd + asset_b_usd
let loan_amount_usd = loan_amount * loan_exchange_rate
let loan_to_value = (loan_amount_usd / total_collateral) * 100.0

printf "LOAN DETAILS\n"
printf "Loan Amount: %f USD\n" loan_amount_usd
printf "Asset A Collateral: %f USD\n" asset_a_usd
printf "Asset B Collateral: %f USD\n" asset_b_usd
printf "Total Collateral: %f USD\n" total_collateral
printf "Loan to Value: %f %%\n\n" loan_to_value


// calculate loanable amount and minimum collateral
let max_loanable_amount = calculate_max_loanable_amount (asset_a_usd + asset_b_usd)
let max_loan_to_value = 1.0 / minimum_collateral_ratio
let minimum_asset_b_collateral = calculate_minimum_assetb_collateral loan_amount_usd

printf "LOAN LIMITS AND REQUIREMENTS\n"
printf "Max Loanable Amount: %f USD\n" max_loanable_amount
printf "Max Loan to Value: %f %%\n" (max_loan_to_value * 100.0)
printf "Minimum Asset B Collateral: %f USD\n\n" minimum_asset_b_collateral

// calculate eligibility
let collateral_ratio = calculate_collateral_ratio asset_a_collateral asset_a_exchange_rate asset_b_collateral asset_b_exchange_rate loan_amount loan_exchange_rate
let eligible = can_loan asset_a_collateral asset_a_exchange_rate asset_b_collateral asset_b_exchange_rate loan_amount loan_exchange_rate loan_term

printf "LOAN ELIGIBILITY\n"
printf "Collateral Ratio: %f\n" collateral_ratio
printf "Minimum Collateral Ratio: %f\n" minimum_collateral_ratio
printf "Eligible: %b\n" eligible

LOAN DETAILS
Loan Amount: 120.000000 USD
Asset A Collateral: 175.000000 USD
Asset B Collateral: 10.000000 USD
Total Collateral: 185.000000 USD
Loan to Value: 64.864865 %

LOAN LIMITS AND REQUIREMENTS
Max Loanable Amount: 92.500000 USD
Max Loan to Value: 50.000000 %
Minimum Asset B Collateral: 24.000000 USD

LOAN CONDITIONS
Minimum Collateral Ratio Met: false
Minimum Collateral Asset B Met: false
Minimum Loan Amount Met: true
Loan Term Met: true

LOAN ELIGIBILITY
Collateral Ratio: 1.541667
Minimum Collateral Ratio: 2.000000
Eligible: false
